<a href="https://colab.research.google.com/github/gellwanger/nerdcast_analysis/blob/main/Nercast_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re

## Import Selenium

In [ ]:
%%capture

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

## Get API

In [ ]:
url = 'https://jovemnerd.com.br/nerdcast/'

get_api = requests.get(url)

text = get_api.text

## Load Soup

In [ ]:
soup = bs(text,  features='html.parser')

## Load Selenium

In [ ]:
driver = webdriver.Chrome(options=options)

driver.get(url)

driver.find_element(By.CLASS_NAME, 'text-button-cookie').click()

## Load List

In [ ]:
list = []
list2 = []
list3 = []
list_duration = []
list_launch = []
list_link = []

## Load Page


In [ ]:
driver.find_element(By.ID, 'show-more-podcasts').click()

time.sleep(10)

new_html = driver.page_source

soup = bs(new_html, 'html.parser')

## Loop


In [ ]:
i = 0

while i < 50:
    button = driver.find_element(By.ID, 'show-more-podcasts')
    style_value = button.get_attribute('style')


    button.click()
    time.sleep(5)
    new_html = driver.page_source
    soup = bs(new_html, 'html.parser')

    i += 1
    print(f'Iteração {i}')

    if i >= 2:
      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

      get_podcast = soup.find_all('a', class_='cat-item product-nerdcast')

      get_podcast_name = soup.find_all('h2', class_='title')

      get_podcast_duration = soup.find_all('div', class_='info')

      list.extend(get_podcast)
      list2.extend(get_podcast_name)
      list3.extend(get_podcast_duration)

      print(f'Coleta concluída, com o arquivo {list[-1].text}!')
      break

## Data Output

In [ ]:
df = pd.DataFrame(columns=['Podcast', 'Link'])

In [ ]:
for item in list:
  df.loc[df.shape[0]] = [item.text.strip(), item['href']]

In [ ]:
df = df[df['Podcast'] != '2011 - 2012 - 2022 - 2023']
df = df[df['Podcast'] != '2017 - 2020']
df = df[df['Podcast'] != '2013 - 2016']


In [ ]:
df.insert(0, 'ID', 1)
df['ID'] = range(1, len(df) + 1)

In [ ]:
df2 = pd.DataFrame(columns=['Assunto', 'Link'])

In [ ]:
for item in list2:
  df2.loc[df2.shape[0]] = [item.text.strip(), item.find('a')['href']]

In [ ]:
list_of_numbers = []

for item in df['Podcast']:

  this = any(char.isdigit() for char in item)

  if this is False:
    item.index
    result_string = ''.join(item)

    list_of_numbers.append(result_string)

  else:
    pass

In [ ]:
df2.insert(0, 'ID', 1)
df2['ID'] = range(1, len(df2) + 1)

In [ ]:
for item in list_of_numbers:
  df = df[df['Podcast'] != item]

In [ ]:
list_text = []


for item in df['Podcast']:
  this = re.split(r'[0-9]', item)

  list_text.extend(this)

In [ ]:
print(list_text)

In [ ]:
list_number = []

for item in df['Podcast']:
  this = re.split(r'[a-z A-Z á-ú]', item)

  list_number.extend(this)

In [ ]:
text_new = []

text_new = [item.strip() for item in list_text if item != '']

In [ ]:
print(text_new)

In [ ]:
number_new = []

number_new = [item for item in list_number if item != '']

In [ ]:
df.insert(len(df.columns) - 1, column = 'Numero', value = 0)

In [ ]:
df['Numero'] = number_new
df['Podcast'] = text_new

In [377]:
df3 = pd.DataFrame(columns=['Duracao', 'Data_Lancamento', 'Link'])

In [373]:
for item in list3[:9]:
  if item.find('time'):
    duracao = item.find('time').get_text().split('•')[0][1:].strip()
    list_duration.append(duracao)

    data_lancamento = item.find('time').get_text().split('•')[1][1:].strip()
    list_launch.append(data_lancamento)

    link = item.find('a')['href']
    list_link.append(link)

  else:
    pass

print(list_duration)
print(list_launch)
print(list_link)

['38 minutos', '58 minutos', '1 minuto', '57 minutos']
['7 de novembro de 2023', '4 de novembro de 2023', '3 de novembro de 2023', '3 de novembro de 2023']
['https://jovemnerd.com.br/nerdcast/la-do-bunker/foi-mal-tava-doidao/', 'https://jovemnerd.com.br/nerdcast/caneca-de-mamicas/so-sei-que-nasa-sei/', 'https://jovemnerd.com.br/nerdcast/bastidores-de-um-jogo-aaa/', 'https://jovemnerd.com.br/nerdcast/nerdtech/o-que-esperar-dos-novos-smartphones/']


In [378]:
df3['Duracao'] = list_duration
df3['Data_Lancamento'] = list_launch
df3['Link'] = list_link

In [379]:
df3.head()

,Duracao,Data_Lancamento,Link
0,38 minutos,7 de novembro de 2023,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,58 minutos,4 de novembro de 2023,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,1 minuto,3 de novembro de 2023,https://jovemnerd.com.br/nerdcast/bastidores-d...
3,57 minutos,3 de novembro de 2023,https://jovemnerd.com.br/nerdcast/nerdtech/o-q...
